In [49]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Link Prediction - Scientometric

Data:
2011-2015, 2011-2016, 2011-2017,...2011-2019

In [21]:
import pandas as pd
import os
import numpy as np

In [22]:
path = os.getcwd()
scientometric_coauthor_path = path+'\\data\\Scientometrics-3\\co-author\\'
scientometric_cooccur_path = path+'\\data\\Scientometrics-3\\co-occurance\\'
jasist_coauthor_path = path+'\\data\\JASIST\\co-author\\'
jasist_cooccur_path = path+'\\data\\JASIST\\co-occurrance\\'

In [56]:
def list_files_paths(files_path):
    files = os.listdir(files_path)
    network_file_paths = []
    mapfile_paths = []
    for file in files:
        print(files_path+file+'\\network.txt')
        print(files_path+file+'\\map.txt')
        network_file_paths.append(files_path+file+'\\network.txt')
        mapfile_paths.append(files_path+file+'\\map.txt')
    return files, network_file_paths, mapfile_paths

files, network_file_paths, mapfile_paths = list_files_paths(jasist_coauthor_path)

C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2015\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2015\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2016\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2016\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2017\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2017\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2018\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2018\map.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2019\network.txt
C:\Users\Liser\Desktop\linchengwei_link_prediction\data\JASIST\co-author\jasist_2010-2019\map.txt


In [58]:
def file_transform(file):
    source_list, target_list, weight_list = [], [], []
    with open(file, 'r', encoding='utf-8') as f:
        f = f.read()
        for i, line in enumerate(f.split('\n')):
            for j, ele in enumerate(line.split('\t')):
                if i == 0:
                    ele = ele.replace('\ufeff', '')
                if j == 0:
                    source_list.append(ele)
                elif j== 1:
                    target_list.append(ele)
                elif j == 2:
                    weight_list.append(ele)
    source_list = source_list[:-1]
    return source_list, target_list, weight_list

def to_df(source_list, target_list, weight_list):
    
    df = pd.DataFrame(columns = ['source', 'target', 'weight'])
    df['source'] = source_list
    df['target'] = target_list
    df['weight'] = weight_list

    return df

In [59]:
df_map_list = []
for f in mapfile_paths:
    df_iter = pd.read_csv(f, sep='\t')
    df_map_list.append(df_iter)

In [60]:
df_edges = []
for f in network_file_paths:
    source_list, target_list, weight_list = file_transform(f)
    df_iter = to_df(source_list, target_list, weight_list)
    df_edges.append(df_iter)

In [61]:
print("File number \nnode:{} \t edge:{}".format(len(df_map_list), len(df_edges)))

File number 
node:5 	 edge:5


##### node_list, containing 7 nodes of different time segments

In [62]:
# all nodes, 2001-2010
def generate_node(df):
    N = list(set(list(set(df['source']))+list(set(df['target']))))
    nodes = []
    for n in N:
        nodes.append(int(n))
    nodes = sorted(nodes)
    print('num of nodes:{}'.format(len(nodes)))
    
    return nodes
node_list = []
for df in df_edges:
    nodes = generate_node(df)
    nodes = list(map(str, nodes))
    node_list.append(nodes)

num of nodes:838
num of nodes:1015
num of nodes:1056
num of nodes:1073
num of nodes:1088


## Generate .net file

    *network Informetrics1990-2004
    *vertices 632
    632 "Shailendra, K"
    *edges
    1 402 1

In [ ]:
def generate_netfiles(files, df_nodes, df_edges, node_list):
    netfile_names = []
    file_name_pref = input('INPUT FILE NAME PREFIX:\t') # scientometrics_coauthor_, JASIST_coauthor_
    for file in files:

        #print("{}".format(file_name_pref)+file)
        netfile_names.append("{}".format(file_name_pref)+file)

    # scientometric_data_2010-2014
    for file_index, netfile_name in enumerate(netfile_names):
        f = open(os.getcwd()+"\\output\\netfiles\\origin\\{}.net".format(netfile_name), "w", encoding='utf-8')
        f.write("*{}\n".format(netfile_name))

        f.write("*vertices {}\n".format(len(node_list[file_index])))
        df_node = df_nodes[file_index]
        for i in range(len(node_list[file_index])):
            label = df_node[df_node['id']==int(node_list[file_index][i])]['label'].values[0]
            f.write('{} "{}"\n'.format(node_list[file_index][i], label))
    #     print("node written")
        f.write('*edges\n')
        for i in range(len(df_edges[file_index])):
            source = int(df_edges[file_index].loc[i][0])
            target = int(df_edges[file_index].loc[i][1])
            weight = int(df_edges[file_index].loc[i][2])
            f.write("{} {} {}\n".format(source, target, weight))
        f.close()
        print('--'*15, netfile_name, '--'*15)
        
generate_netfiles(jasist_files, df_nodes_jasist, df_edges_jasist, node_list_jasist)
generate_netfiles(scient_files, df_nodes_scient, df_edges_scient, node_list_scient)

    *network Informetrics1990-2004
    *vertices 632
    632 "Shailendra, K"
    *edges
    1 402 1

In [37]:
len(node_list[4])

3690

In [39]:
df_map

,id,label,x,y,cluster,weight<Links>,weight<Total link strength>,weight<Documents>,weight<Citations>,weight<Norm. citations>,score<Avg. pub. year>,score<Avg. citations>,score<Avg. norm. citations>
0,1,"aarssen, lonnie w.",-0.3113,-0.3972,73,3,3,1,11,0.6568,2013.0,11.0,0.6568
1,2,"abad-garcia, maria-francisca",0.0102,0.2632,23,3,3,1,29,2.4378,2015.0,29.0,2.4378
2,3,"abatemarco, antonio",0.0444,-1.1463,384,1,1,1,2,0.1194,2013.0,2.0,0.1194
3,4,"abbas, ash mohammad",0.5476,-1.0420,585,0,0,1,20,0.7928,2011.0,20.0,0.7928
4,5,"abbasi, ali",-0.9792,-0.2322,114,4,4,1,26,1.0307,2011.0,26.0,1.0307
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,2922,"zuo, wei",-0.3402,1.0708,192,3,3,1,6,0.3582,2013.0,6.0,0.3582
2922,2923,"zurbruegg, ralf",-1.0469,0.2198,489,1,1,1,3,0.1961,2014.0,3.0,0.1961
2923,2924,"zych, izabela",0.4709,-0.5508,584,1,1,1,57,2.9369,2012.0,57.0,2.9369
2924,2925,"zyczkowski, karol",0.4634,0.0767,744,0,0,1,17,0.5162,2010.0,17.0,0.5162


In [63]:
netfile_names = []
file_name_pref = input('INPUT FILE NAME:\t') # scientometrics_coauthor_, JASIST_coauthor_
for file in files:
    
    print("{}".format(file_name_pref)+file)
    netfile_names.append("{}".format(file_name_pref)+file)

INPUT FILE NAME:	JASIST_coauthor_
JASIST_coauthor_jasist_2010-2015
JASIST_coauthor_jasist_2010-2016
JASIST_coauthor_jasist_2010-2017
JASIST_coauthor_jasist_2010-2018
JASIST_coauthor_jasist_2010-2019


In [43]:
len(node_list)

5

In [64]:
# scientometric_data_2010-2014
for file_index, netfile_name in enumerate(netfile_names):
    f = open(os.getcwd()+"\\output\\netfiles\\{}.net".format(netfile_name), "w", encoding='utf-8')
    f.write("*{}\n".format(netfile_name))

    f.write("*vertices {}\n".format(len(node_list[file_index])))
    df_map = df_map_list[file_index]
    for i in range(len(node_list[file_index])):
        label = df_map[df_map['id']==int(node_list[file_index][i])]['label'].values[0]
        f.write('{} "{}"\n'.format(node_list[file_index][i], label))
#     print("node written")
    f.write('*edges\n')
    for i in range(len(df_edges[file_index])):
        source = int(df_edges[file_index].loc[i][0])
        target = int(df_edges[file_index].loc[i][1])
        weight = int(df_edges[file_index].loc[i][2])
        f.write("{} {} {}\n".format(source, target, weight))
    f.close()
    print('--'*15, netfile_name, '--'*15)

------------------------------ JASIST_coauthor_jasist_2010-2015 ------------------------------
------------------------------ JASIST_coauthor_jasist_2010-2016 ------------------------------
------------------------------ JASIST_coauthor_jasist_2010-2017 ------------------------------
------------------------------ JASIST_coauthor_jasist_2010-2018 ------------------------------
------------------------------ JASIST_coauthor_jasist_2010-2019 ------------------------------


In [310]:
# with open(os.getcwd()+"\\examples\\inf1990-2004.net.txt", 'r', encoding='utf-8') as f1:
#     f1 = f1.read()
# f1.split('\n')

## Link Prediction Package

### Applied
0630

In [1]:
import linkpred
import os
path = os.getcwd()
G = linkpred.read_network(path+"\\output\\netfiles\\scientometrics_coauthor_2010-2017.net")
len(G)

3476

In [3]:
simrank = linkpred.predictors.SimRank(G, excluded=G.edges())

In [4]:
simrank_results = simrank.predict(c=0.5)

In [5]:
top = simrank_results.top(5)

In [6]:
for authors, score in top.items():
    print(authors, score)

yap, xiao-shan - goh, kim-leng 0.5
wu, shengnan - wang, ju 0.5
wu, shengnan - chen, bikun 0.5
wray, k. brad - vanclay, jerome k. 0.5
wray, k. brad - haunschild, robin 0.5


# Cosine Similarity

### find subgraph, nodes that overlap

In [14]:
for i in range(len(node_list)):
    print('t{}'.format(i), len(node_list[i]), 'nodes')

t0 2766 nodes
t1 3266 nodes
t2 3476 nodes
t3 3604 nodes
t4 3690 nodes


###### itertools generate pairs

In [60]:
import itertools
pairs = []
for pair in itertools.combinations(list(range(5)),2): # 
    pairs.append(pair)

In [61]:
pairs

[(0, 1),
 (0, 2),
 (0, 3),
 (0, 4),
 (1, 2),
 (1, 3),
 (1, 4),
 (2, 3),
 (2, 4),
 (3, 4)]

In [16]:
print(pairs[0][0])
print(pairs[0][1])

0
1


In [17]:
n1 = node_list[pairs[0][0]]
n2 = node_list[pairs[0][1]]
intersect_n12 = list(set(n1).intersection(set(n2)))
print('Intersection node numbers: ', len(intersect_n12))

Intersection node numbers:  2633


### intersection df

In [18]:
df1 = df_list[pairs[0][0]]
df2 = df_list[pairs[0][1]]

In [19]:
df1_intersect = df1[df1['source'].isin(intersect_n12) & df1['target'].isin(intersect_n12)==True]
df2_intersect = df2[df2['source'].isin(intersect_n12) & df2['target'].isin(intersect_n12)==True]
print('df1 \t before: {} edges, \n\t after:  {} edges'.format(len(df1), len(df1_intersect)))
print('--'*20)
print('df2 \t before: {} edges, \n\t after:  {} edges'.format(len(df2), len(df2_intersect)))

df1 	 before: 4771 edges, 
	 after:  4281 edges
----------------------------------------
df2 	 before: 5771 edges, 
	 after:  3736 edges


In [68]:
intersect_n12[-1]

'2926'

In [63]:
df1_intersect

,source,target,weight
0,1,279,1
1,1,1365,1
2,1,1551,1
3,2,163,1
4,2,842,1
...,...,...,...
4765,2872,2875,3
4766,2873,2875,4
4767,2874,2875,1
4768,2883,2891,1


#### create_empty_matrix

In [69]:
intersect_n12 = list(map(int, intersect_n12))
intersect_n12 = sorted(intersect_n12)
intersect_n12 = list(map(str, intersect_n12))

In [22]:
import numpy as np
def create_empty_matrix(nodes):
    t = pd.DataFrame(np.zeros((len(nodes), len(nodes))))
    t.columns = nodes
    t.index = nodes
    return t

t1_0 = create_empty_matrix(intersect_n12) # t1, direct
t1_1 = create_empty_matrix(intersect_n12) # t1, cosine similarity
t2_0 = create_empty_matrix(intersect_n12) # t2, direct
t2_1 = create_empty_matrix(intersect_n12) # t2, cosine similarity

In [23]:
t1_0

,1,2,3,5,6,7,8,9,10,11,...,2915,2916,2917,2918,2920,2921,2922,2923,2924,2926
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2921,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2922,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2924,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Add weight

In [24]:
def add_weight(df, t):
    # add weight
    for i in intersect_n12:
        # print(i)
        A = df['source']==i
        for k in range(len(df[A]['target'])):
            j = df[A]['target'].iloc[k]
            weight = df[A]['weight'].iloc[k]
            # print(i, j, weight)
            # print(df[A]['target'].iloc[k], df[A]['weight'].iloc[k])
            t.at[i, j] = int(weight)
            t.at[j, i] = int(weight)
    return t
t1_0 = add_weight(df1_intersect, t1_0) # t1, direct
t2_0 = add_weight(df2_intersect, t2_0) # t2, direct

In [25]:
t1_0[t1_0['1']==1]

,1,2,3,5,6,7,8,9,10,11,...,2915,2916,2917,2918,2920,2921,2922,2923,2924,2926
279,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1365,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1551,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
t2_0[t2_0['1']==1]

,1,2,3,5,6,7,8,9,10,11,...,2915,2916,2917,2918,2920,2921,2922,2923,2924,2926
333,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1596,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Generate Cosine Similarity

In [27]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
def generate_cosine_sim(t):
    t_sparse = sparse.csr_matrix(t)
    similarities = cosine_similarity(t_sparse)
    # remove self related
    def remove_self_related(similarities):
        for i in range(len(similarities)):
            for j in range(len(similarities)):
                if i==j:
                    similarities[i][j] = 0
        return similarities
    similarities = remove_self_related(similarities)
    
    return similarities

t1_1 = generate_cosine_sim(t1_0)
t2_1 = generate_cosine_sim(t2_0)

In [29]:
df_t1_1_cosine = pd.DataFrame(t1_1)
df_t2_1_cosine = pd.DataFrame(t2_1)
def transform_to_df(similarities, intersect_n12):
    df = pd.DataFrame(similarities)
    df.columns = intersect_n12
    df.index = intersect_n12
    return df
df_t1_1_cosine = transform_to_df(t1_1, intersect_n12)
df_t2_1_cosine = transform_to_df(t2_1, intersect_n12)

In [43]:
df_t1_1_cosine[df_t1_1_cosine['1']!=0]

,1,2,3,5,6,7,8,9,10,11,...,2915,2916,2917,2918,2920,2921,2922,2923,2924,2926
58,0.235702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
279,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1365,0.666667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1551,0.516398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1914,0.235702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
